In [2]:
#for streamlit debug

In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
import time
import tensorflow as tf
from altair.vegalite.v4.schema.core import Header
import altair as alt
import serial
from PIL import Image


N_of_readings = 100 #10Hz=100ms : 10 -> 1sec

classes_values = ['beer',
                  'coffee',
                  'orange']

#this is only for demo
classes_values_12 = ['beer',
                      'bread',
                      'carrot',
                      'cheese',
                      'coffee',
                      'cola',
                      'cucumber',
                      'wein',
                      'lemon',
                      'orange',
                      'sprite',
                      'tomato']

## Data to predict
test_sample = [2378, 64, 15, 64, 93, 15, 66]

## collect data to predict
# N_of_readings = 10 #5Hz=200ms : 10 -> 2sec
serial_port = '/dev/ttyACM0'
baud_rate = 9600

GGS_list = []

## Simulations
##----------------------------
time_ms    = np.linspace(start=test_sample[0], stop=test_sample[0]+10000, num=N_of_readings, dtype=int)
#normal(loc=0.0, scale=1.0, size=None)
B102NO2    = np.random.normal(loc=test_sample[1], scale=test_sample[1]/30, size=N_of_readings)
B302C2H5OH = np.random.normal(loc=test_sample[2], scale=test_sample[2]/30, size=N_of_readings)
B502VOC    = np.random.normal(loc=test_sample[3], scale=test_sample[3]/30, size=N_of_readings)
B702CO     = np.random.normal(loc=test_sample[4], scale=test_sample[4]/30, size=N_of_readings)
TdegC      = np.random.normal(loc=test_sample[5], scale=test_sample[5]/30, size=N_of_readings)
RH         = np.random.normal(loc=test_sample[6], scale=test_sample[6]/30, size=N_of_readings)

st.session_state['GGS_df'] = pd.DataFrame([time_ms, 
        B102NO2,
        B302C2H5OH,
        B502VOC,
        B702CO,
        TdegC,
        RH], 
        index=["time_ms", "NO2", 
                 "C2H5OH", "VOC", "CO",
                  "Temperature", "Humidity"]).T





if st.session_state['GGS_df'].shape == (N_of_readings, 7):

    data2predictArr = st.session_state['GGS_df'].iloc[:,1:].to_numpy()
    data2predictArr.shape=(data2predictArr.shape[0],6)
    predictionN = model_ff.predict(data2predictArr)
    print(predictionN)
    data2predictArrAver = predictionN.mean(axis=0)
    print(data2predictArrAver)


    predictionN = model_ff.predict(data2predictArr)
    #Plot
    chart_data = pd.DataFrame(
       predictionN.transpose(),
       index=classes_values,
       )
    data = pd.melt(chart_data.reset_index(), id_vars=["index"])
    chart = (
        alt.Chart(data)
        .mark_bar()
        .encode(
            x=alt.X("value", type="quantitative", title="Probability"),
            y=alt.Y("index", type="nominal", title="Category"),
            #color=alt.Color("variable", type="nominal", title=""),
            order=alt.Order("variable", sort="descending"),
            ).properties(title='Predictions'
            ).properties(width=100, height=305)
        )
    st.altair_chart(chart, use_container_width=True)
